## Tarefa 2

### 1.Monte um passo a passo para o algoritmo RF

#### Primeiro passo: Bootstrap + Feature Selection.
#### Bootstrap: Uma quantidade de conjunto de dados pré determinados serão criados, esses novos conjuntos terão a mesma quantidade de linhas que o conjunto original e para cada linha será copiado aleatóriamente uma linha do conjunto original, podendo repetir os valores. 
#### Feature Selection: Bem parecido com o bootstrap, porém ao invés de selecionar as linhas, agora as colunas vão ser selecionadas aleatóriamente, o número de colunas vai ser igual a:
#### Para casos de classificação
#### m = &radic;p  
#### Para casos de regressão
#### m = p / 3
#### Onde P é igual a quantidade de features do conjunto original e m o número de features usadas na criação dos novos conjuntos do Random Forest. Essa é uma quantidade estipulada por Leo Breiman autor do processo. <br>

#### Segundo passo: Árvores
#### Na segunda etapa vão ser criadas árvores para cada conjunto criado na etapa anterior, as etapas vão fazer seus passos individualmente, crescendo até que atinja a profundidade máxima pré definida e vão dar seus resultados finais. <br>

#### Terceiro passo: Agregação
#### Para finalizar, os resultados obtidos na etapa anterior vão ser consolidados, para casos de classificação a moda será o resultado final e para casos de regressão a média será o resultado final.

#### 2. Explique com suas palavras o Randomforest
#### O random forest é um algoritimo de aprendizado de máquina que faz o uso de bootstrap e feature selection para seleção de conjunto de dados e aplica o modelo de árvore para cada conjunto criado, isso ajuda a evitar overfitting devido a aleatoriedade na captação dos dados e do uso das features, então quanto mais árvores criadas mais conjuntos com caracteristicas diferentes e melhor a generalização. 

#### 3.Qual a diferença entre Bagging e Random Forest?
#### Bagging e Random Forest são técnicas ensemble, que combinam vários modelos para melhorar seus resultados. 
#### Na etapa de bootstrap o bagging seleciona todas as features, porém o random forest seleciona um número pré determinado de features, isso pode ajudar a diminuir a correlação entre as variáveis.<br>
#### As árvores do bagging não tem seus hiperparametros alterados, já no random forest a árvore é podada deixando uma profundida menor ou outros limitadores, isso ajuda em uma melhor generalização.<br>
#### Em resumo são a mesma coisa, porém o random forest como o próprio nome sugere adiciona uma pitata de aleatoriedade nas suas etapas para evitar overfitting e melhorar sua generalização.


#### 4. (Opcional) Implementar em python o Random Forest–Bootstrap–Featureselection–Modelagem com Decisiontrees–Agregação

In [16]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [2]:
df = pd.read_csv('demo01.csv')
df

,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,possui_celular,possui_fone_comercial,possui_fone,possui_email,qt_pessoas_residencia,mau
0,M,Y,Y,0,Working,Secondary / secondary special,Married,House / apartment,58.832877,3.106849,1,0,0,0,2.0,False
1,F,N,Y,0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,52.356164,8.358904,1,0,1,1,1.0,False
2,F,N,Y,0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,52.356164,8.358904,1,0,1,1,1.0,False
3,M,Y,Y,0,Working,Higher education,Married,House / apartment,46.224658,2.106849,1,1,1,1,2.0,False
4,F,Y,N,0,Working,Incomplete higher,Married,House / apartment,29.230137,3.021918,1,0,0,0,2.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16645,F,N,Y,0,Working,Secondary / secondary special,Civil marriage,House / apartment,54.109589,9.884932,1,0,0,0,2.0,True
16646,F,N,Y,0,Commercial associate,Secondary / secondary special,Married,House / apartment,43.389041,7.380822,1,1,1,0,2.0,True
16647,M,Y,Y,0,Working,Secondary / secondary special,Married,House / apartment,30.005479,9.800000,1,1,0,0,2.0,True
16648,M,Y,Y,0,Working,Secondary / secondary special,Married,House / apartment,30.005479,9.800000,1,1,0,0,2.0,True


In [3]:
# Verificando valores faltantes
missing_values = df.isnull().sum()
missing_values

sexo                     0
posse_de_veiculo         0
posse_de_imovel          0
qtd_filhos               0
tipo_renda               0
educacao                 0
estado_civil             0
tipo_residencia          0
idade                    0
tempo_emprego            0
possui_celular           0
possui_fone_comercial    0
possui_fone              0
possui_email             0
qt_pessoas_residencia    0
mau                      0
dtype: int64

In [4]:
# Verificando o tipo das features
df.dtypes

sexo                      object
posse_de_veiculo          object
posse_de_imovel           object
qtd_filhos                 int64
tipo_renda                object
educacao                  object
estado_civil              object
tipo_residencia           object
idade                    float64
tempo_emprego            float64
possui_celular             int64
possui_fone_comercial      int64
possui_fone                int64
possui_email               int64
qt_pessoas_residencia    float64
mau                         bool
dtype: object

In [5]:
# Seleciona as colunas que vão ser transformadas em dummies
features_to_encode = df.select_dtypes(include=['object'])
features_to_encode

,sexo,posse_de_veiculo,posse_de_imovel,tipo_renda,educacao,estado_civil,tipo_residencia
0,M,Y,Y,Working,Secondary / secondary special,Married,House / apartment
1,F,N,Y,Commercial associate,Secondary / secondary special,Single / not married,House / apartment
2,F,N,Y,Commercial associate,Secondary / secondary special,Single / not married,House / apartment
3,M,Y,Y,Working,Higher education,Married,House / apartment
4,F,Y,N,Working,Incomplete higher,Married,House / apartment
...,...,...,...,...,...,...,...
16645,F,N,Y,Working,Secondary / secondary special,Civil marriage,House / apartment
16646,F,N,Y,Commercial associate,Secondary / secondary special,Married,House / apartment
16647,M,Y,Y,Working,Secondary / secondary special,Married,House / apartment
16648,M,Y,Y,Working,Secondary / secondary special,Married,House / apartment


In [6]:
# Seleciona as colunas que já estão prontas, para juntar depois
features_ready = df.select_dtypes(exclude=['object'])
features_ready

,qtd_filhos,idade,tempo_emprego,possui_celular,possui_fone_comercial,possui_fone,possui_email,qt_pessoas_residencia,mau
0,0,58.832877,3.106849,1,0,0,0,2.0,False
1,0,52.356164,8.358904,1,0,1,1,1.0,False
2,0,52.356164,8.358904,1,0,1,1,1.0,False
3,0,46.224658,2.106849,1,1,1,1,2.0,False
4,0,29.230137,3.021918,1,0,0,0,2.0,False
...,...,...,...,...,...,...,...,...,...
16645,0,54.109589,9.884932,1,0,0,0,2.0,True
16646,0,43.389041,7.380822,1,1,1,0,2.0,True
16647,0,30.005479,9.800000,1,1,0,0,2.0,True
16648,0,30.005479,9.800000,1,1,0,0,2.0,True


In [7]:
# Transformando tudo em dummies, zero ou um
features_encoded = pd.get_dummies(features_to_encode)
features_encoded

,sexo_F,sexo_M,posse_de_veiculo_N,posse_de_veiculo_Y,posse_de_imovel_N,posse_de_imovel_Y,tipo_renda_Commercial associate,tipo_renda_Pensioner,tipo_renda_State servant,tipo_renda_Student,...,estado_civil_Married,estado_civil_Separated,estado_civil_Single / not married,estado_civil_Widow,tipo_residencia_Co-op apartment,tipo_residencia_House / apartment,tipo_residencia_Municipal apartment,tipo_residencia_Office apartment,tipo_residencia_Rented apartment,tipo_residencia_With parents
0,0,1,0,1,0,1,0,0,0,0,...,1,0,0,0,0,1,0,0,0,0
1,1,0,1,0,0,1,1,0,0,0,...,0,0,1,0,0,1,0,0,0,0
2,1,0,1,0,0,1,1,0,0,0,...,0,0,1,0,0,1,0,0,0,0
3,0,1,0,1,0,1,0,0,0,0,...,1,0,0,0,0,1,0,0,0,0
4,1,0,0,1,1,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16645,1,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
16646,1,0,1,0,0,1,1,0,0,0,...,1,0,0,0,0,1,0,0,0,0
16647,0,1,0,1,0,1,0,0,0,0,...,1,0,0,0,0,1,0,0,0,0
16648,0,1,0,1,0,1,0,0,0,0,...,1,0,0,0,0,1,0,0,0,0


In [9]:
# Juntando os DataFrames em um dataframe final
df_encoded = pd.concat([features_encoded, features_ready], axis=1)
df_encoded

,sexo_F,sexo_M,posse_de_veiculo_N,posse_de_veiculo_Y,posse_de_imovel_N,posse_de_imovel_Y,tipo_renda_Commercial associate,tipo_renda_Pensioner,tipo_renda_State servant,tipo_renda_Student,...,tipo_residencia_With parents,qtd_filhos,idade,tempo_emprego,possui_celular,possui_fone_comercial,possui_fone,possui_email,qt_pessoas_residencia,mau
0,0,1,0,1,0,1,0,0,0,0,...,0,0,58.832877,3.106849,1,0,0,0,2.0,False
1,1,0,1,0,0,1,1,0,0,0,...,0,0,52.356164,8.358904,1,0,1,1,1.0,False
2,1,0,1,0,0,1,1,0,0,0,...,0,0,52.356164,8.358904,1,0,1,1,1.0,False
3,0,1,0,1,0,1,0,0,0,0,...,0,0,46.224658,2.106849,1,1,1,1,2.0,False
4,1,0,0,1,1,0,0,0,0,0,...,0,0,29.230137,3.021918,1,0,0,0,2.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16645,1,0,1,0,0,1,0,0,0,0,...,0,0,54.109589,9.884932,1,0,0,0,2.0,True
16646,1,0,1,0,0,1,1,0,0,0,...,0,0,43.389041,7.380822,1,1,1,0,2.0,True
16647,0,1,0,1,0,1,0,0,0,0,...,0,0,30.005479,9.800000,1,1,0,0,2.0,True
16648,0,1,0,1,0,1,0,0,0,0,...,0,0,30.005479,9.800000,1,1,0,0,2.0,True


In [13]:
# Separando as variáveis explicativas e target
X = df_encoded.drop('mau', axis=1)
y = df_encoded['mau']

# Separando em conjunto de dados de treino e test, 25% para teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

In [17]:
# Criando o classificador
rf_clf = RandomForestClassifier(n_estimators = 100, random_state = 1)

# Treinando com os valores de treino
rf_clf.fit(X_train, y_train)

# Fazendo predição dos valores, agora com os dados de teste
y_pred = rf_clf.predict(X_test)

# Salvando o resultado de acurácia
acurracy = accuracy_score(y_test, y_pred)
acurracy

0.9759788613980303

#### Essa seleção foi feita com o RandomForest padrão, ainda podendo aplicar alterações nos hyperparametros e fazer validação cruzada.